In [77]:
import torch
from matplotlib import pyplot as plt
import pandas as pd
import os

In [78]:
base_path = '..'
# Gets label
csv_path = os.path.join(base_path,'data/3DYoga90_corrected.csv')

SAVE_PATH = os.path.join(base_path, 'biomechanical_features')
os.makedirs(SAVE_PATH, exist_ok=True)

# Classification classes
pose_list = ['mountain', 'half-way-lift', 'standing-forward-bend', 'downward-dog']
subset_of_poses = pose_list
NUM_CLASSES = len(pose_list)

dataset_dir = os.path.join(base_path, 'official_dataset')
assert os.path.isdir(dataset_dir), f"Directory '{dataset_dir}' does not exist."

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [79]:
meta_info_path = os.path.join(base_path, 'data')
pose_index = pd.read_csv(f'{meta_info_path}/pose-index.csv')
sequence_index = pd.read_csv(f'{meta_info_path}/3DYoga90_corrected.csv')

In [80]:
# Keep only relevant columns
def read_meta_data():
    meta_info_path = os.path.join(base_path, 'data')
    pose_index = pd.read_csv(f'{meta_info_path}/pose-index.csv')
    sequence_index = pd.read_csv(f'{meta_info_path}/3DYoga90_corrected.csv')
    parquet_index = sequence_index[['sequence_id', 'l3_pose', 'split']]
    return parquet_index

In [81]:

import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class Yoga3DDataset(Dataset):
    def __init__(self, parquet_index, root_dir =  dataset_dir,subset_of_poses= subset_of_poses, sub_sampling_length = 20, transform=None, max_frames=None):
        self.parquet_index = parquet_index
        self.parquet_index = self.parquet_index[self.parquet_index['l3_pose'].isin(subset_of_poses)]
        self.root_dir = root_dir
        self.transform = transform
        self.max_frames = max_frames
        self.sub_sampling_length = sub_sampling_length
        self.pose_to_label = {pose: i for i, pose in enumerate(subset_of_poses)}
        self.use_augmentation = False

        self.cache = dict()
        self.idx_to_seq = dict()

    def __len__(self):
        return len(self.parquet_index)

    def __getitem__(self, idx):
        if idx in self.cache:
            data, label = self.cache[idx]
        else:
            fname, pose_name, _ = self.parquet_index.iloc[idx]
            label = self.pose_to_label[pose_name]
            path = os.path.join(self.root_dir, f'{fname}.parquet')

            df = pd.read_parquet(path)
            df = df.drop(columns=['frame', 'row_id', 'type','landmark_index'])

            data = self.to_tensor(df)
            # data = self.sub_sample(data)
            data = data.permute(1,0,2)
            self.cache[idx] = (data, label)
            self.idx_to_seq[idx] = fname

        if self.transform and self.use_augmentation:
            data = self.transform(data.clone())

        return data, self.idx_to_seq[idx] # C, T , V

    def sub_sample(self, data):
        # data(Number_of_frames, 3, 33)
        total_frames = data.shape[0]
        indices = torch.linspace(0, total_frames -1 , self.sub_sampling_length, dtype= int)
        return data[indices]

    def to_tensor(self, df):
        # Reshape the data to (num_frames, num_landmarks, 3)  ## WHAT WHAT? this doesn't make sense remove this line you are doing (number of frames, 3 , 33)
        num_frames = len(df) // 33  # Assuming 33 landmarks per frame
        data = df.values.reshape(num_frames, 33, 3)
        return torch.FloatTensor(data).permute(0, 2, 1)

In [82]:
import torch

def calculate_vector(point1, point2):
    """
    Calculate vector between two 3D points
    """
    return point2 - point1

def calculate_angle(vector1, vector2):
    """
    Calculate angle between two 3D vectors using dot product
    Returns angle in degrees
    """
    dot_product = torch.dot(vector1, vector2)
    norms = torch.norm(vector1) * torch.norm(vector2)
    
    # Handle numerical stability
    cos_angle = torch.clamp(dot_product / norms, -1.0, 1.0)
    angle_rad = torch.acos(cos_angle)
    return torch.rad2deg(angle_rad)

def calculate_projected_angle(vector1, vector2, normal):
    """
    Calculate angle between two vectors when projected onto a plane
    defined by its normal vector
    """
    # Project vectors onto the plane
    proj1 = vector1 - torch.dot(vector1, normal) * normal
    proj2 = vector2 - torch.dot(vector2, normal) * normal
    
    return calculate_angle(proj1, proj2)

def calculate_joint_angles(poses):
    """
    Calculate relevant joint angles from pose data
    Input: poses - torch tensor of shape (frames, 33, 3)
    Output: dictionary of joint angles
    """
    joint_configs = {
        # Upper body
        'right_shoulder': {
            'joints': (13, 11, 23),  # right_elbow, right_shoulder, right_hip
            'planes': ['sagittal', 'transverse', 'frontal']
        },
        'left_shoulder': {
            'joints': (14, 12, 24),  # left_elbow, left_shoulder, left_hip
            'planes': ['sagittal', 'transverse', 'frontal']
        },
        'right_elbow': {
            'joints': (11, 13, 15),  # right_shoulder, right_elbow, right_wrist
            'planes': ['sagittal', 'transverse', 'frontal']
        },
        'left_elbow': {
            'joints': (12, 14, 16),  # left_shoulder, left_elbow, left_wrist
            'planes': ['sagittal', 'transverse', 'frontal']
        },
        
        # Lower body
        'right_hip': {
            'joints': (11, 23, 25),  # right_shoulder, right_hip, right_knee
            'planes': ['sagittal', 'transverse', 'frontal']
        },
        'left_hip': {
            'joints': (12, 24, 26),  # left_shoulder, left_hip, left_knee
            'planes': ['sagittal', 'transverse', 'frontal']
        },
        'right_knee': {
            'joints': (23, 25, 27),  # right_hip, right_knee, right_ankle
            'planes': ['sagittal', 'transverse', 'frontal']  # Now including transverse
        },
        'left_knee': {
            'joints': (24, 26, 28),  # left_hip, left_knee, left_ankle
            'planes': ['sagittal', 'transverse', 'frontal']  # Now including transverse
        },
        'right_ankle': {
            'joints': (25, 27, 31),  # right_knee, right_ankle, right_foot_index
            'planes': ['sagittal', 'transverse', 'frontal']
        },
        'left_ankle': {
            'joints': (26, 28, 32),  # left_knee, left_ankle, left_foot_index
            'planes': ['sagittal', 'transverse', 'frontal']
        }
    }
    num_frames = poses.shape[0]
    angles = {}
    angles_tensor = torch.zeros((num_frames,len(joint_configs),1 ))
    # Define joint triplets for angle calculation
    
    # Define anatomical planes using normal vectors
    planes = {
        'sagittal': torch.tensor([1, 0, 0]),  # Left-right axis (flexion/extension)
        'frontal': torch.tensor([0, 0, 1]),   # Forward-backward axis (abduction/adduction)
        'transverse': torch.tensor([0, 1, 0])  # Up-down axis (internal/external rotation)
    }
    
    # Calculate angles for each frame
    for frame in range(num_frames):
        frame_angles = {}
        
        for j, (joint_name, config) in enumerate(joint_configs.items()):
            j1, j2, j3 = config['joints']
            
            # Calculate vectors
            vector1 = calculate_vector(poses[frame, j2], poses[frame, j1])
            vector2 = calculate_vector(poses[frame, j2], poses[frame, j3])
            
            # Calculate 3D angle
            computed_angle = calculate_angle(vector1, vector2)
            frame_angles[f"{joint_name}_3d"] = computed_angle 
            angles_tensor[frame, j,0] =  computed_angle
            # # Calculate projected angles on anatomical planes
            # for plane_name in config['planes']:
            #     normal = planes[plane_name]
            #     projected_angle = calculate_projected_angle(vector1, vector2, normal)
            #     frame_angles[f"{joint_name}_{plane_name}"] = projected_angle
        
        angles[frame] = frame_angles
    
    return angles, angles_tensor


In [83]:
import torch
import numpy as np
from typing import Dict, List, Tuple

class BiomechanicalFeatureExtractor:
    def __init__(self):
        """
        Initialize the feature extractor.
        Note: Simplified version using frame-by-frame differences
        """
        pass
    
    def compute_velocities(self, joint_positions: torch.Tensor) -> torch.Tensor:
        """
        Compute joint velocities from positions using simple frame differences.
        
        Args:
            joint_positions: Tensor of shape (frames, num_joints, 3)
            
        Returns:
            Tensor of shape (frames, num_joints, 3) containing velocities
        """
        # Initialize velocities tensor with zeros
        velocities = torch.zeros_like(joint_positions)
        
        # Compute velocities as simple differences between consecutive frames
        velocities[:-1] = joint_positions[1:] - joint_positions[:-1]
        
        # For the last frame, use the same velocity as the second-to-last frame
        velocities[-1] = velocities[-2]
        
        return velocities
    
    def compute_accelerations(self, velocities: torch.Tensor) -> torch.Tensor:
        """
        Compute joint accelerations from velocities using simple frame differences.
        
        Args:
            velocities: Tensor of shape (frames, num_joints, 3)
            
        Returns:
            Tensor of shape (frames, num_joints, 3) containing accelerations
        """
        # Initialize accelerations tensor with zeros
        accelerations = torch.zeros_like(velocities)
        
        # Compute accelerations as simple differences between consecutive velocities
        accelerations[:-1] = velocities[1:] - velocities[:-1]
        
        # For the last frame, use the same acceleration as the second-to-last frame
        accelerations[-1] = accelerations[-2]
        
        return accelerations
    

    def compute_joint_angles(
        self, 
        joint_positions: torch.Tensor,
    ) -> Tuple[Dict[str, torch.Tensor], torch.Tensor]:
        joint_angles, angles_tensor = calculate_joint_angles(joint_positions)
        return joint_angles, angles_tensor
    
    def extract_features(
        self, 
        joint_positions: torch.Tensor,
    ) -> Dict[str, torch.Tensor]:
        """
        Extract all biomechanical features from joint positions.
        
        Args:
            joint_positions: Tensor of shape (frames, num_joints, 3)
        Returns:
            Dictionary containing all computed features
        """
        velocities = self.compute_velocities(joint_positions)
        accelerations = self.compute_accelerations(velocities)
        angles_dict, angles = self.compute_joint_angles(joint_positions)# Access Angles Dict if you need to know which scalar value corresponds to which angle

        return {
            "Joint Position": joint_positions,
            "Joint Angles": angles,
            "Joint Velocity": velocities,
            "Joint Acceleration": accelerations
        }
    
    @staticmethod
    def save_features(features: Dict[str, torch.Tensor], filename: str):
        """Save features to a .pt file."""
        torch.save(features, filename)
        print(f"saving {filename}")
    
    @staticmethod
    def load_features(filename: str) -> Dict[str, torch.Tensor]:
        """Load features from a .pt file."""
        return torch.load(f"{filename}.pt")


In [84]:
import os
from tqdm import tqdm

def process_dataset(
    dataset: Yoga3DDataset,
    output_dir: str,
) -> None:
    """
    Process entire dataset and save biomechanical features.
    
    Args:
        dataset: Yoga3DDataset instance
        output_dir: Directory to save extracted features
        joint_triplets: List of joint triplet indices for angle calculation
    """
    
    extractor = BiomechanicalFeatureExtractor()
    
    for i in tqdm(range(len(dataset)), desc="Processing sequences"):
        joints, fname = dataset[i]  # Shape: (3, frames, 33) 
        # From: (3, frames, 33) -> To: (frames, 33, 3)
        joint_positions = joints.permute(1, 2, 0)
        # Extract features
        features = extractor.extract_features(
            joint_positions
        )

        output_path = os.path.join(output_dir, f"{fname}.pt")
        extractor.save_features(features, output_path)

if __name__ == "__main__":
    dataset = Yoga3DDataset(read_meta_data())
   
    process_dataset(
        dataset=dataset,
        output_dir=SAVE_PATH,
    )

Processing sequences:   0%|          | 3/844 [00:00<00:36, 23.33it/s]

saving ..\biomechanical_features\1000.pt
saving ..\biomechanical_features\1002.pt
saving ..\biomechanical_features\1003.pt
saving ..\biomechanical_features\1004.pt
saving ..\biomechanical_features\1005.pt


Processing sequences:   1%|          | 8/844 [00:00<00:45, 18.31it/s]

saving ..\biomechanical_features\1006.pt
saving ..\biomechanical_features\1007.pt
saving ..\biomechanical_features\1008.pt
saving ..\biomechanical_features\1009.pt


Processing sequences:   2%|▏         | 13/844 [00:00<00:51, 16.10it/s]

...

Processing sequences:   8%|▊         | 70/844 [00:05<01:43,  7.45it/s]

saving ..\biomechanical_features\1070.pt
saving ..\biomechanical_features\1071.pt


Processing sequences:   9%|▊         | 73/844 [00:05<01:00, 12.75it/s]

saving ..\biomechanical_features\1072.pt
saving ..\biomechanical_features\1073.pt
saving ..\biomechanical_features\1074.pt


KeyboardInterrupt: 

In [27]:

# import os
# import torch
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
# from mpl_toolkits.mplot3d import Axes3D

# # Define the MediaPipe Pose landmarks and their connections
# # Landmark names and their corresponding indices
# LANDMARK_NAMES = [
#     'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner',
#     'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left',
#     'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
#     'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index',
#     'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip',
#     'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel',
#     'right_heel', 'left_foot_index', 'right_foot_index'
# ]

# # Define connections between landmarks
# POSE_CONNECTIONS = [
#     (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8),
#     (1, 2), (2, 3), (4, 5), (5, 6), (7, 9), (8, 10), (11, 12),
#     (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (12, 14),
#     (14, 16), (16, 18), (16, 20), (16, 22), (11, 23), (12, 24),
#     (23, 24), (23, 25), (24, 26), (25, 27), (26, 28), (27, 29),
#     (28, 30), (29, 31), (30, 32)
# ]

# def visualize_pose_sequence(data, label, connections=POSE_CONNECTIONS, figsize=(8, 8)):
#     """
#     Visualize a sequence of 3D poses.

#     Parameters:
#         data (torch.Tensor): Tensor of shape (channels, number_of_frames, landmarks=33)
#                              where channels are x, y, z coordinates.
#         label (int): The label corresponding to the pose.
#         connections (list of tuples): Landmark connections to draw the skeleton.
#         figsize (tuple): Figure size for the plot.
#     """
#     # Convert tensor to numpy array
#     data_np = data.numpy()
#     # Assuming channels are in the order x, y, z
#     x = data_np[0]  # shape: (number_of_frames, 33)
#     y = data_np[1]
#     z = data_np[2]
#     # print(x.shape)
#     num_frames = x.shape[0]

#     # Create a figure and a 3D subplot
#     fig = plt.figure(figsize=figsize)
#     ax = fig.add_subplot(111, projection='3d')
#     plt.title(f'Pose: {label}')

#     # Function to update the plot for each frame
#     def update(frame):
#         ax.cla()
#         ax.set_xlim3d(-1, 1)
#         ax.set_ylim3d(-1, 1)
#         ax.set_zlim3d(-1, 1)
#         ax.set_xlabel('X')
#         ax.set_ylabel('Y')
#         ax.set_zlabel('Z')
#         plt.title(f'Pose: {label}, Frame: {frame + 1}/{num_frames}')

#         # Scatter plot of landmarks
#         ax.scatter(x[frame], y[frame], z[frame], c='r', marker='.')

#         # Draw connections
#         for connection in connections:
#             idx1, idx2 = connection
#             ax.plot([x[frame, idx1], x[frame, idx2]],
#                     [y[frame, idx1], y[frame, idx2]],
#                     [z[frame, idx1], z[frame, idx2]], 'b-')
#         # plt.show() 
#         return

#     # Create the animation
#     anim = FuncAnimation(fig, update, frames=num_frames, interval=100)
#     plt.show()
#     save_dir = 'data'
#     os.makedirs(save_dir, exist_ok=True)
#     save_path = os.path.join(save_dir, f'test_{label}.mp4')
#     anim.save(save_path)